In [56]:
import random
import pandas as pd

# --- Constants ---
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
    '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11
}

# True count zones (based on betting thresholds)
true_count_zones = {
    "Highly Unfavourable": lambda tc: tc <= 0,
    "Unfavourable": lambda tc: tc == 1,
    "Neutral": lambda tc: tc == 2,
    "Favourable": lambda tc: tc == 3,
    "Highly Favourable": lambda tc: tc >= 4
}

# Basic Strategy Tables
hard_totals = {
    5:  ['H'] * 10, 6: ['H'] * 10, 7: ['H'] * 10, 8: ['H'] * 10,
    9:  ['H','D','D','D','D','H','H','H','H','H'],
    10: ['D']*8 + ['H','H'],
    11: ['D']*9 + ['H'],
    12: ['H','H','S','S','S','H','H','H','H','H'],
    13: ['S']*5 + ['H']*5,
    14: ['S']*5 + ['H']*5,
    15: ['S']*5 + ['H']*4 + ['SR','H'],
    16: ['S']*5 + ['H']*2 + ['SR','SR','SR'],
    17: ['S'] * 10, 18: ['S'] * 10, 19: ['S'] * 10,
    20: ['S'] * 10, 21: ['S'] * 10
}
soft_totals = {
    13: ['H','H','H','D','D','H','H','H','H','H'],
    14: ['H','H','H','D','D','H','H','H','H','H'],
    15: ['H','H','D','D','D','H','H','H','H','H'],
    16: ['H','H','D','D','D','H','H','H','H','H'],
    17: ['H','D','D','D','D','H','H','H','H','H'],
    18: ['S','D','D','D','D','S','S','H','H','H'],
    19: ['S'] * 10, 20: ['S'] * 10, 21: ['S'] * 10
}
pairs = {
    4:  ['P','P','P','P','P','P','H','H','H','H'],
    6:  ['P','P','P','P','P','P','H','H','H','H'],
    8:  ['H','H','H','P','P','H','H','H','H','H'],
    10: ['P','P','P','P','P','H','H','H','H','H'],
    12: ['P','P','P','P','P','P','H','H','H','H'],
    14: ['P','P','P','P','P','P','H','H','H','H'],
    16: ['P'] * 10,
    18: ['P','P','P','P','P','S','P','P','S','S'],
    20: ['S'] * 10,
    22: ['P'] * 10
}

# --- Classes ---
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = ranks[rank]

class Deck:
    def __init__(self, num_decks=6, penetration=0.75):
        self.num_decks = num_decks
        self.penetration = penetration
        self.full_deck = [Card(suit, rank) for suit in suits for rank in ranks] * num_decks
        self.shuffle()

    def shuffle(self):
        self.cards = self.full_deck.copy()
        random.shuffle(self.cards)
        self.running_count = 0

    def deal(self):
        reshuffle_point = int((1 - self.penetration) * len(self.full_deck))
        if len(self.cards) <= reshuffle_point:
            self.shuffle()
        card = self.cards.pop()
        self.update_count(card)
        return card

    def update_count(self, card):
        if card.rank in ['10', 'J', 'Q', 'K', 'A']:
            self.running_count -= 1
        elif card.rank in ['2', '3', '4', '5', '6']:
            self.running_count += 1

    def true_count(self):
        decks_remaining = len(self.cards) / 52
        return round(self.running_count / decks_remaining) if decks_remaining > 0 else 0

# --- Strategy Logic ---
def hand_value(hand):
    value = sum(card.value for card in hand)
    aces = sum(1 for card in hand if card.rank == 'A')
    while value > 21 and aces:
        value -= 10
        aces -= 1
    return value

def is_pair(hand):
    return len(hand) == 2 and hand[0].rank == hand[1].rank

def is_soft_hand(hand):
    has_ace = any(card.rank == 'A' for card in hand)
    return has_ace and hand_value(hand) != sum(card.value for card in hand)

def get_upcard_index(card):
    if card.rank in ['10', 'J', 'Q', 'K']: return 8
    if card.rank == 'A': return 9
    return int(card.rank) - 2

def get_player_action(hand, dealer_card):
    upcard_index = get_upcard_index(dealer_card)
    value = hand_value(hand)
    if is_pair(hand):
        pair_total = hand[0].value * 2
        return pairs.get(pair_total, ['H']*10)[upcard_index]
    elif is_soft_hand(hand):
        return soft_totals.get(value, ['H']*10)[upcard_index]
    else:
        return hard_totals.get(value, ['H']*10)[upcard_index]

def get_bet_from_true_count(tc):
    if tc <= 0: return 1
    elif tc == 1: return 1
    elif tc == 2: return 1
    elif tc == 3: return 4
    else: return 8

# --- Game Logic ---
def play_hand_with_strategy(deck):
    player = [deck.deal(), deck.deal()]
    dealer = [deck.deal(), deck.deal()]
    dealer_upcard = dealer[0]

    while True:
        action = get_player_action(player, dealer_upcard)
        if action == 'H':
            player.append(deck.deal())
            if hand_value(player) > 21:
                break
        elif action == 'D':
            if len(player) == 2:
                player.append(deck.deal())
            break
        elif action == 'S':
            break
        elif action == 'P':
            player.append(deck.deal())  # Simplified split
            break
        elif action == 'SR':
            return "Loss"

    while hand_value(dealer) < 17:
        dealer.append(deck.deal())

    pv, dv = hand_value(player), hand_value(dealer)
    if pv > 21: return "Loss"
    elif dv > 21: return "Win"
    elif pv > dv: return "Win"
    elif pv < dv: return "Loss"
    return "Draw"

# --- Main Simulation ---
def simulate_blackjack_with_betting(num_decks=6, num_hands=10000, penetration=0.75):
    deck = Deck(num_decks=num_decks, penetration=penetration)
    results, bankroll = [], 0

    for _ in range(num_hands):
        true_count = deck.true_count()
        bet = get_bet_from_true_count(true_count)
        outcome = play_hand_with_strategy(deck)

        # Bankroll effect
        if outcome == "Win":
            bankroll += bet
        elif outcome == "Loss":
            bankroll -= bet

        for zone, condition in true_count_zones.items():
            if condition(true_count):
                results.append({
                    "True Count": true_count,
                    "Count Zone": zone,
                    "Outcome": outcome,
                    "Bet": bet,
                    "Bankroll": bankroll,
                    "Profit": bet if outcome == "Win" else (-bet if outcome == "Loss" else 0)
                })
                break

    return pd.DataFrame(results)

In [60]:
df = simulate_blackjack_with_betting(num_decks=6, num_hands=500000)
df

,True Count,Count Zone,Outcome,Bet,Bankroll,Profit
0,0,Highly Unfavourable,Draw,1,0,0
1,0,Highly Unfavourable,Draw,1,0,0
2,0,Highly Unfavourable,Loss,1,-1,-1
3,0,Highly Unfavourable,Loss,1,-2,-1
4,0,Highly Unfavourable,Win,1,-1,1
...,...,...,...,...,...,...
499995,-1,Highly Unfavourable,Win,1,-72423,1
499996,-2,Highly Unfavourable,Draw,1,-72423,0
499997,-2,Highly Unfavourable,Loss,1,-72424,-1
499998,-2,Highly Unfavourable,Win,1,-72423,1


In [61]:
df['Profit'] = df['Profit']  # Already in results

summary = (
    df.groupby("Count Zone")
    .agg(
        Games=('Outcome', 'count'),
        Total_Bet=('Bet', 'sum'),
        Net_Profit=('Profit', 'sum'),
        Win_Rate=('Outcome', lambda x: (x == "Win").mean())
    )
)
summary['ROI %'] = (summary['Net_Profit'] / summary['Total_Bet'] * 100).round(2)
summary

,Games,Total_Bet,Net_Profit,Win_Rate,ROI %
Count Zone,,,,,
Favourable,24542,98168,-8440,0.412273,-8.60
Highly Favourable,31321,250568,-19224,0.416015,-7.67
Highly Unfavourable,318804,318804,-33210,0.405528,-10.42
Neutral,44191,44191,-4047,0.409337,-9.16
Unfavourable,81142,81142,-7501,0.410047,-9.24
